In [ ]:
from dotenv import load_dotenv
import os
from pypdf import PdfReader
import gradio as gr
from openai import OpenAI

In [ ]:
load_dotenv(override=True)
openai = OpenAI()
os.getenv("OPENAI_API_KEY")

In [ ]:
reader = PdfReader("me/linkedin.pdf")
linkedin_text = ""
for page in reader.pages:
    linkedin_text += page.extract_text()

print(linkedin_text[:400])


In [ ]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary_text = f.read()

In [ ]:
name = "Ed Donner"

In [ ]:
system_prompt = f"""
You are acting as {name}. You are answering questions on {name}'s website, \
particualarly questions related to {name}'s background, experience, and skills. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so. Don't make up an answer.
"""

In [ ]:
system_prompt += f"\n\n## Summary\n{summary_text}\n\n## LinkedIn Profile\n{linkedin_text}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

gr.ChatInterface(chat, type="messages").launch()

In [ ]:
# Ask LLM to evaluate answer

from pydantic import BaseModel

class Evaluation(BaseModel):
    isAcceptable: bool
    feedback: str


In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary_text}\n\n## LinkedIn Profile:\n{linkedin_text}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
deepseek = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="http://localhost:11434/v1",
)


In [ ]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = deepseek.beta.chat.completions.parse(
        model="deepseek-r1:7b",
        messages=messages,
        response_format=Evaluation
    )
    return response.choices[0].message.parsed

In [ ]:
!ollama pull deepseek-r1:7b

In [ ]:
request = deepseek.chat.completions.create(
    model="deepseek-r1:7b",
    messages=[{"role": "user", "content": "do you hold a patent?"}]
)


In [ ]:
request.choices[0].message.content

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)
print(response.choices[0].message.content)



In [ ]:

def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:

def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.isAcceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:

gr.ChatInterface(chat, type="messages").launch()